In [1]:
# import packages
import pandas as pd
import numpy as np
import pyblp
import statsmodels.api as sm
from linearmodels.iv import IV2SLS
from scipy.optimize import minimize
import scipy
from numba import jit, njit, prange
import time
import multiprocessing as mp
import pickle
from sklearn.linear_model import LinearRegression

# Data Description

- The data is about tht automotive market in the U.S.
- It has 2217 observations, and 16 variables
  - 20 markets
  - 26 firms
  - 20 years: from 1971 to 1990
 
- Key variables:

|Variable name |Description|
|----|----------------|
|mpg |tens of miles per gallon (also indicating miles per dollar)|
|hpwt | the ratio of horsepower to weight (in HP per 10 lbs.)|
|air |whether the car has air conditioning|
|quantity |in unit of 1000|
|price |in \$1000 units|
|size |length times width|
|share |🙋? market size was measured using the number of households in the U.S.|
|share_out |🙋? |

In [2]:
# import data (raw car data)
df = pd.read_csv(r'../data/BLP_1995_data/BLP_1995_data.csv')
print('number of observations: ', df.shape[0], ';', 'number of variables:', df.shape[1])
df.head()

number of observations:  2217 ; number of variables: 16


,prodvec,modelvec,newmodv,model_year,id,firmid,market,hpwt,space,air,mpd,price,mpg,quantity,share,share_out
0,AMGREM,AMGREM,AMGREM71,71,129,15,1,0.528997,1.1502,0.0,1.888146,4.935802,1.697,70.096,0.001051,0.880106
1,AMHORN,AMHORN,AMHORN71,71,130,15,1,0.494324,1.2780,0.0,1.935989,5.516049,1.740,44.678,0.000670,0.880106
2,AMJAVL,AMJAVL,AMJAVL71,71,132,15,1,0.467613,1.4592,0.0,1.716799,7.108642,1.543,22.705,0.000341,0.880106
3,AMMATA,AMMATA,AMMATA71,71,134,15,1,0.426540,1.6068,0.0,1.687871,6.839506,1.517,34.821,0.000522,0.880106
4,AMAMBS,AMAMBS,AMAMBS71,71,136,15,1,0.452489,1.6458,0.0,1.504286,8.928395,1.352,29.499,0.000442,0.880106


In [3]:
# obtain additional instrumental data from pyblp.data
# product_data = pd.read_csv(pyblp.data.BLP_PRODUCTS_LOCATION)
# product_data.head()

In [4]:
# clean data and create new variables

df[["ln_hpwt", "ln_space", "ln_mpg", "ln_mpd", "ln_price"]] = \
    df[["hpwt", "space", "mpg", "mpd", "price"]].apply(lambda x: np.log(x))

# instrument
df["trend"] = df["market"] + 70

df["cons"] = 1

df["s_0"] = np.log(1 - df.share.groupby(df["model_year"]).transform("sum"))

df["s_i"] = np.log(df.share)
df["dif"] = df.s_i - df.s_0
df["dif_2"] = np.log(df.share) - np.log(df.share_out)
df["ln_price"] = np.log(df.price)

df.head()

,prodvec,modelvec,newmodv,model_year,id,firmid,market,hpwt,space,air,...,ln_space,ln_mpg,ln_mpd,ln_price,trend,cons,s_0,s_i,dif,dif_2
0,AMGREM,AMGREM,AMGREM71,71,129,15,1,0.528997,1.1502,0.0,...,0.139936,0.528862,0.635595,1.596515,71,1,-0.171483,-6.858013,-6.686531,-6.730300
1,AMHORN,AMHORN,AMHORN71,71,130,15,1,0.494324,1.2780,0.0,...,0.245296,0.553885,0.660618,1.707662,71,1,-0.171483,-7.308233,-7.136750,-7.180520
2,AMJAVL,AMJAVL,AMJAVL71,71,132,15,1,0.467613,1.4592,0.0,...,0.377888,0.433729,0.540462,1.961311,71,1,-0.171483,-7.983628,-7.812146,-7.855915
3,AMMATA,AMMATA,AMMATA71,71,134,15,1,0.426540,1.6068,0.0,...,0.474245,0.416735,0.523468,1.922716,71,1,-0.171483,-7.557843,-7.386360,-7.430130
4,AMAMBS,AMAMBS,AMAMBS71,71,136,15,1,0.452489,1.6458,0.0,...,0.498227,0.301585,0.408318,2.189237,71,1,-0.171483,-7.724201,-7.552718,-7.596488


In [5]:
# prepare mediate data

# demand variables
X = df[["cons", "hpwt", "air", "mpd", "space"]].values

# suppy variables
W = df[["cons", "ln_hpwt", "air", "ln_mpg", "ln_space", "trend"]].values

# price
Price = df.price.values

# initial delta_0 estimate: log(share) - log(share outside good)
Delta_0 = df.dif_2.values

# number of goods per market
#J = df.groupby("model_year").sum().cons.values
J = df.groupby("market").sum().cons.values

# number of draws per market
N = 500

# number of markets
T = len(J)

# estimated log income means for years 1971 - 1990
IncomeMeans = [2.01156, 2.06526, 2.07843, 2.05775, 2.02915, 2.05346, 2.06745,
               2.09805, 2.10404, 2.07208, 2.06019, 2.06561, 2.07672, 2.10437, 
               2.12608, 2.16426, 2.18071, 2.18856, 2.21250, 2.18377]

# standard deviation of log incomes, assuming empirically given in 1995
sigma_v = 1.72

# number of terms that have the random coefficient
# according to table 4, they are constant, hpwt, air, mpd, space, price
k = 5

# markets for ijt
markets = df.market.values

# unique markets
unique_markets = np.unique(df.market)

# firms
firms = np.reshape(df.firmid.values, (-1,1))

# Review on Discrete Choice Model
- The indirect utility of consumer $i$ from consuming product $i$ in market $t$: 
$$u_{ijt}=\alpha_i (y_i-p_{jt}) + {x}_{jt}^{T}{\beta}_i+\xi_{jt}+ \epsilon_{ijt}$$
  -  $\alpha_i$: consumer $i$’s marginal utility from income
  -  $y_i$: the income of consumer $i$
  - $p_{jt}$: price of product $j$ in market $t$
  - ${x}_{jt}$: K-dimensional (column) vector of non-price attributes of product $j$ in market $t$
  - ${\beta}_i$: K-dimensional (column) vector of individual-specific taste coefficients
  - $\xi_{jt}$: utility of unobserved attributes of product $j$ in market $t$
  - $\varepsilon_{njm}$: idiosyncratic unobserved utility



# 1 Standard (Simple) Logit Model 

- Assuming $\epsilon_{ijt}$ follows Type I Extreme Value Distribution, then the probability that consumer $i$ will choose product $j$ 
$$p_{ijt}(v_i)=\frac{exp(\delta_{jt}+v_{ijt})}{exp(\delta_{0}+v_{ikt})+\sum_{k=1}^J exp(\delta_{kt}+v_{ikt})}$$
  - $\delta_{jt}$: mean utility
  - $p_{ijt}(v_i)$: the probability of customer $i$ selecting product $j$ in market $t$

- If $v_{ijt}=0$, it reduces to the standard logit model , i.e., $$p_{ijt}(v_i)=\frac{exp(\delta_{jt} )}{exp(\delta_{0})+\sum_{k=1}^J exp(\delta_{kt})}$$
  - Then, $$ln(p_{jt}) - ln(p_{0}) = \delta_{jt} - \delta_{0}$$
  - market share $s_{jt} = p_{jt}$
  - WLG, assume $\delta_{0}$ to be 0, and the utility $\delta_{jt} = ln(s_{jt}) - ln(s_{0})$

##  Demand Side: Regression Specification of Standard (Simple) Logit Model 
- Outcome varible: $\delta_{jt}$, i.e., $ln(s_{jt}) - ln(s_{0})$
- Formula:
>$\delta_{jt}$ ~ hpwt + air + mpd + space + price

Most coefficients are of the expected sign, although the (imprecisely estimated) negative coefficients on air conditioning and size are anomalies, as one would expect these attributes to yield positive marginal utility. On the other hand these estimates have a distinctly implausible set of implications on own price elasticities. **The estimated coefficient on price in Table III implies that 1494 of the 2217 models have inelastic demands.(🙋?)** This is inconsistent with profit maximizing price choices. Moreover this is not simply a problem generated by an imprecise estimate of the price coefficient.

In [6]:
# Table 3 column (1) in BLP 1995
df["utility_simple_logit"] = np.log(df.share) - np.log(df.share_out)
ols_res = sm.OLS.from_formula('utility_simple_logit ~ 1 + hpwt + air + mpd + space + price', data = df).fit()
print(ols_res.summary2(float_format="%.6f"))

                   Results: Ordinary least squares
Model:              OLS                  Adj. R-squared:     0.386    
Dependent Variable: utility_simple_logit AIC:                6648.6879
Date:               2021-08-20 13:07     BIC:                6682.9114
No. Observations:   2217                 Log-Likelihood:     -3318.3  
Df Model:           5                    F-statistic:        279.3    
Df Residuals:       2211                 Prob (F-statistic): 5.83e-232
R-squared:          0.387                Scale:              1.1716   
------------------------------------------------------------------------
             Coef.     Std.Err.      t       P>|t|     [0.025     0.975]
------------------------------------------------------------------------
Intercept   -10.0730     0.2528   -39.8458   0.0000   -10.5688   -9.5773
hpwt         -0.1231     0.2771    -0.4442   0.6570    -0.6666    0.4204
air          -0.0344     0.0728    -0.4728   0.6364    -0.1771    0.1083
mpd           

# 2 Using Intrumental Variable to Deal With Endogeneity

These abnormal elasticities are due to the endogeneity of price. That is, cars with better unobserved quality will tend to have higher prices as well (producers know unobserved characteristics but schalors don't). A simple remedy would be to instrument for price. 
- Any factors that are correlated with the price (quantity), but are *not correlated with the demand (supply) disturbance*, $\xi$ ($\omega$), will be appropriate instruments for the demand (supply) equation
- Since both price and unobserved characteristics enter the demand function in *a nonliear form*, traditional instrumental variablbe methods cannot be straighforwardly used

BLP propose using three sets of instruments
1. The observed product characteristics (which are assumed orthogonal to the unobserved characteristics)
1. The sum of product characteristics for all models in a given market.
1. The sum of product characteristics for all models marketed by a single firm in a given market.

*In BLP (1995), they actually say that when calculating the second set of instruments one should exclude the product you are calculating the instruments for, i.e. sum over product characteristics of all other products sold by the firm. A similar rule is used for the third set, i.e. just sum over its competitor’s models. I have not seen this used consistently in literature and in fact BLP doesn’t seem to do this in Table III of their paper so I stick to the three sets laid out above. Shapiro and Gentzkow also found a mistake in how BLP calculated their instruments. They multiply each product characteristic by the number of models the firm sells in each market rather than sum across the characteristics. I follow this mistaken calculation so as to match BLP’s original results.




Set 1
$$x_{kt}$$

Set 2
- In oligopoly, the price of a good in a market depends on the market structure, i.e., what kind of products are available in the market
- For example, if there are similar products in the market, the price will tend to be lower
- Then, the product characteristics of other products in the market , will be valid instrument for the price of goods in a given market (affect its price, but not affect its demand beyond the influence of its price), $p_{jt}$ 
- Thus, IV uses
$$\sum_{k \in J_t} x_{kt}$$  
  - $J_t$ is the set of products that are available in market  
  - $f$ is the firm that owns product $j$ and $F_f$ is the set of products firm $f$ owns
  - Correction: $\sum_{k\neq j, k \in J_t\setminus F_t} x_{kt}$, where $J_t\setminus F_t$ is the set of products in market $t$ except the products owned by firm $f$
  
Set 3
- If there are multi-product firms, whether the other good is owned by the same company will also affect the price
- Thus, IV uses
$$\sum_{k\neq j, k \in J_t\cap F_t} x_{kt}$$
  - Corresction: $\sum_{k\neq j, k \in J_t\cap F_t} x_{kt}$, where $J_t\cap F_t$ is the set of products firm $f$ owns in market $t$ 

In [7]:
# generate instrumental variable
# parameters
# # col_charateristic: product characteristics except prices
# # type_instrument = 'market': the sum of product characteristics for all models in a given market
# # type_instrument = 'firm': the sum of product characteristics for all models marketed by a single firm in a given market
# # col_market: colname of markets
# # col_firm: colname of firms

def generate_instrument(data, col_charateristic, type_instrument, col_market, col_firm = None):
    data = pd.DataFrame(data)
    output = None

    if type_instrument == 'market':
        key_columns = [col_market]
        for col in col_charateristic:
            key_columns.append(col)
            
        by_market = data[key_columns].groupby(col_market).agg('sum').reset_index()
        output = data[[col_market]].merge(by_market, how = 'left', on = col_market)
        output = output[col_charateristic]
        
    elif type_instrument == 'firm':
        if col_firm is None:
            print('col_firm must be speficified.')
        else:
            key_columns = [col_market, col_firm]
            for col in col_charateristic:
                key_columns.append(col)
                
            by_firm = data[key_columns].groupby([col_market, col_firm]).agg('sum').reset_index()
            output = data[[col_market, col_firm]].merge(by_firm, how = 'left', on = [col_market, col_firm])
            output = output[col_charateristic]
    else:
        print('Only two types of instrments are acceptable: market, firm.')
        
    return output

In [8]:
generate_instrument(df, ['cons', 'hpwt', 'air', 'mpd', 'space'], 'market', 'market').head()

,cons,hpwt,air,mpd,space
0,92,46.925499,0.0,176.05818,132.7013
1,92,46.925499,0.0,176.05818,132.7013
2,92,46.925499,0.0,176.05818,132.7013
3,92,46.925499,0.0,176.05818,132.7013
4,92,46.925499,0.0,176.05818,132.7013


In [9]:
generate_instrument(df, ['cons', 'hpwt', 'air', 'mpd', 'space'], 'firm', 'market', 'firmid').head()

,cons,hpwt,air,mpd,space
0,5,2.369963,0.0,8.733091,7.14
1,5,2.369963,0.0,8.733091,7.14
2,5,2.369963,0.0,8.733091,7.14
3,5,2.369963,0.0,8.733091,7.14
4,5,2.369963,0.0,8.733091,7.14


Recall IV estimator as $$\hat{{\beta}}_{IV}=(R'W'X)^{-1}R'W'Y$$
- $W$: a $n\times j$ array of variables $W$
  1. These variables are uncorrelated with $U$; we say in this case that these instruments are clean. 
  2. The matrix of correlations between the variables in $X$ and the variables in $W$ is of maximum possible rank (= $k$); we say in this case that these instruments are fully correlated. Call the instruments proper if they satisfy 1 and 2.
- $R$: a $k\times k$ weighting matrix that we get to choose
- If there are exactly as many instruments as there are explanatory variables, j = k, then the IV estimator is uniquely determined, $\hat{\beta}_{IV}=(W'X)^{-1}W'Y$, and $R$ is irrelevant
- However, if $j>k$, each $R$ determines a different IV estimator.


In [10]:
# IV estimator

Z = np.hstack((X, generate_instrument(df, ['cons', 'hpwt', 'air', 'mpd', 'space'], 'firm', 'market', 'firmid').values, \
               generate_instrument(df, ['cons', 'hpwt', 'air', 'mpd', 'space'], 'market', 'market').values))

baseData = np.hstack((X, Price.reshape((-1,1)))) # X

zxw1 = Z.T @ baseData # Z'X

bx1 = np.linalg.inv(zxw1.T @ zxw1)@ zxw1.T @ Z.T @ Delta_0 # (Z'X)'Z'X(Z'X)'Z'delta_0, i.e., R = Z'X

e = Delta_0 - baseData @ bx1

g_ind = e.reshape((-1,1)) * Z # GMM estimator E[Z*error]=0

demean = g_ind - g_ind.mean(axis=0).reshape((1,-1))

vg = demean.T @ demean / demean.shape[0]

w0 = np.linalg.inv(vg)

t3c2 = np.linalg.inv(zxw1.T @ w0 @ zxw1) @ zxw1.T @ w0 @ Z.T @ Delta_0 # (Z'X)'w0Z'X (Z'X)'w0Z'delta_0, i.e., R = w0'Z'X
t3c2

array([-9.91644978,  1.11575606,  0.79441418,  0.18455798,  2.48584052,
       -0.15747768])

# 3 Supply Side: a Cobb-Douglas Form
- Marginal cost (mc) is assumed to take a Cobb-Douglas form, i.e.
$$ \textit{mc} = \gamma_1^{w_1}\gamma_2^{w_2}...\gamma_k^{w_k}e^{\epsilon} $$

- Taking logs of both sides gives the linear form

- Formula:
> ln_price ~ ln_hpwt + air + ln_mpg + ln_space + trend

In [11]:
# Table 3 column (3) in BLP 1995
ols_res = sm.OLS.from_formula('ln_price ~ 1 + ln_hpwt + air + ln_mpg + ln_space + trend', data = df).fit()
print(ols_res.summary2(float_format="%.6f"))

                 Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.656    
Dependent Variable: ln_price         AIC:                1146.0122
Date:               2021-08-20 13:07 BIC:                1180.2356
No. Observations:   2217             Log-Likelihood:     -567.01  
Df Model:           5                F-statistic:        844.9    
Df Residuals:       2211             Prob (F-statistic): 0.00     
R-squared:          0.656            Scale:              0.097915 
-------------------------------------------------------------------
                Coef.   Std.Err.     t     P>|t|    [0.025   0.975]
-------------------------------------------------------------------
Intercept       1.8819    0.1188  15.8465  0.0000   1.6490   2.1148
ln_hpwt         0.5203    0.0351  14.8327  0.0000   0.4515   0.5891
air             0.6798    0.0188  36.2471  0.0000   0.6430   0.7165
ln_mpg         -0.4706    0.0485  -9.6943  0.0000  -0.5658  -0.3754
ln_spa

# Random Coefficient

In [12]:
class delta:
    def __init__(self, delta):
        self.delta = delta

# initialize a delta object using the delta_0 values
d = delta(Delta_0)

In [16]:
# set seed
np.random.seed(1128)

# N: simulations
# k: number of terms that have the random coefficient
# T: years
m_t = np.repeat(IncomeMeans, N) # T years * N simulations

# matrix of simulated values
# # each raw is one simulation
# # last column is price/income
# # note that BLP treat it as the same individuals in the market across all years (Nevo 2000)

# different draws for each market
V = np.reshape(np.random.standard_normal((k + 1) * N * T), (T * N, k + 1))

# income if we have different draws per market
y_it = np.exp(m_t + sigma_v * V[:, k]).reshape(T,N).T # N simulations * T years

# initial parameter guess (from BLP(1995))
theta_2 = [3.612, 4.628, 1.818, 1.050, 2.056, 43.501]

The indirect utility of consumer $i$ from consuming product $i$ in market $t$:
$$u_{ijt}=\alpha_i y_i + [-p_{jt}, {x}_{jt}^{T}]\begin{bmatrix}\alpha_i \\ {\beta}_i\end{bmatrix}+\xi_{jt}+ \epsilon_{ijt}$$
$$=\alpha_i y_i + \delta_{jt} + {[-p_{jt}, {x}_{jt}^{T}](\Pi D_i+\Sigma v_i)} +  \epsilon_{ijt}$$


In [18]:
# the loops that calculate utility in a separate function so that it can be
# @jit: stop python when python is not inappropriate; run in parallel. 
# inputs:
# # x: matrix of demand characteristics
# # v: monte carlo draws of N simulations
# # p: price vector
# # y: income of individuals
# # delta: guess for the mean utility
# # theta_2: non-linear params (sigma - can think of as stdev's)
# # J: vector of number of goods per market
# # T: numer of markets
# # N: number of simulations
# outputs:
# # a (moldes * simulations) matrix; each entry is a utility

# @jit(nopython = True, parallel = True) 
def util_iter(out, x, v, p, y, delta, theta_2, J, T, N):
    # first iterate over the individuals 
    for i in prange(N): # simulation i in N
        # iterator through market t and product j
        tj = 0
        # iterate over the markets
        for t in prange(T): # market t
            # market size of market t; number of products in market t
            mktSize = J[t]
            # income for individual i (simulation i) in market t
            y_im = y[i, t]
            
            # iterate over goods in a particular market
            # 'cons', 'hpwt', 'air', 'mpd', 'space', 'price'
            for j in prange(mktSize):
                out[tj, i] = delta[tj] + \
                v[N * t + i, 0] * theta_2[0] * x[tj, 0] + \
                v[N * t + i, 1] * theta_2[1] * x[tj, 1] + \
                v[N * t + i, 2] * theta_2[2] * x[tj, 2] + \
                v[N * t + i, 3] * theta_2[3] * x[tj, 3] + \
                v[N * t + i, 4] * theta_2[4] * x[tj, 4] - \
                theta_2[5] / y_im * p[tj]
                
                tj += 1
    return out

# computes indirect utility given parameters

# @jit(nopython = True, parallel = True) 
def compute_indirect_utility(x, v, p, y, delta, theta_2, J, T, N):
    # make sure theta_2 are positive
    theta_2 = np.abs(theta_2)
    
    # output matrix
    out = np.zeros((np.sum(J), N))
    
    # call the iteration function to calculate utilities
    out = util_iter(out, x, v, p, y, delta, theta_2, J, T, N)
     
    return out

In [21]:
out = compute_indirect_utility(X, V, Price, y_it, Delta_0, theta_2, J, T, N)
print(out.shape)
print(out[0,1])

(2217, 500)
-13.151605123999907


## Function *compute_share*
For each consumer, in each market, it calculates the choice probabilities when given both a vector representing $u_{ijt}$. Note that both of these arguments need to be exponentiated.

In [22]:
# computes the implied shares of goods in each market given inputs
# inputs:
# # x: matrix of demand characteristics
# # v: monte carlo draws of N simulations
# # p: price vector
# # y: income of individuals
# # delta: guess for the mean utility
# # theta_2: non-linear params (sigma - can think of as stdev's)
# # J: vector of number of goods per market
# # T: numer of markets
# # N: number of simulations
# outputs: 
# a (moldes * simulations) matrix; each entry is the quantity each individual purchases of each good in each market
# a (moldes * 1) vector; each model's share

# @jit(nopython = True, parallel = True) 
def compute_share(x, v, p, y, delta, theta_2, J, T, N):
    q = np.zeros((np.sum(J), N))
    
    # obtain vector of indirect utilities
    u = compute_indirect_utility(x, v, p, y, delta, theta_2, J, T, N)
    
    # exponentiate the utilities
    exp_u = np.exp(u)
    
    # pointer to first good in the market
    first_good = 0
            
    for t in prange(T):
        # market size of market t; number of products in market t
        mktSize = J[t]

        # calculate the numerator of the share eq
        numer = exp_u[first_good:first_good + mktSize,:]

        # calculate the denomerator of the share eq
        denom = 1 + numer.sum(axis = 0)    
          
        # calculate the quantity each indv purchases of each good in each market
        q[first_good:first_good + mktSize,:] = numer/denom
        
        first_good += mktSize
    
    # to obtain shares, assume that each simulation carries the same weight.
    # this averages each row, which is essentially computing the sahres for each
    # good in each market. 
    s = np.matmul(q, np.repeat(1/N, N))
    
    return [q,s]

In [25]:
[q,s] = compute_share(X, V, Price, y_it, Delta_0, theta_2, J, T, N)
print(q.shape, s.shape)
print(q[0,0], s[0])

(2217, 500) (2217,)
0.0004849639412537921 0.0005520443234300488


## Function *solve_delta*: Use Contraction Mapping to Get Delta

The contraction mapping outlined in BLP is given by

$$ \delta_{t+1} = \delta_{t} + \ln s_{act} - \ln s_{est}(\delta_t) $$

From a computational stand point, Nevo (2000) recommends using the transformation

$$ e^{\delta_{t+1}} = e^{\delta_{t}} \frac{s_{act}}{s_{est}(\delta_t)} $$

In the following function, I use the BLP (1995) method.

In [28]:
def solve_delta(s, x, v, p, y, delta, theta_2, J, T, N, tol):
    # define the tolerance variable
    eps = 10
    
    # renaming delta as delta^r
    delta_old = delta
    
    while eps > tol:
        # Aviv's step 1: obtain predicted shares and quantities
        q_s = compute_share(x, v, p, y, delta_old, 
                            theta_2, J, T, N)
        
        # extract the shares
        sigma_jt = q_s[1]
        
        # step 2: use contraction mapping to find delta
        delta_new = delta_old + np.log(s/sigma_jt)
        
        # update tolerance
        eps = np.max(np.abs(delta_new - delta_old))
        
        delta_old = delta_new.copy()
    
    return delta_old

## Function *calc_mc* Calculating Marginal Costs
The above code is sufficient if you are only interested in modeling the demand side. However, estimates may be made more precise if a supply side is also incorporated. 

In [29]:
# calculates the marginal costs given probabilities and shares
# q_s: output of compute share, a list of probabilities matrix(q) and shares vector(s)
# firms: vector of firms operating in each market (length is JxT)
# marks: vector of unique markets (length T)
# markets: vector indicating observation in which market (length JxT)

def calc_mc(q_s, firms, p, y, alpha, J, T, N, marks, markets):
    
    # declare output vector
    out = np.zeros((np.sum(J)))
    
    # make sure the value of alpha is positive
    alpha = np.abs(alpha)
    
    # read in quantities
    q = q_s[0]
    
    # read in shares
    s = q_s[1].reshape((-1,1))
    
    # reshape some vectors into column vectors
    p = p.reshape((-1,1))
    
    # iterate over markets
    for m in marks:
        # obtain list of firms operating in that market/year
        firm_yr = firms[markets == m]
        
        # obtain list of prices of goods in that market/year
        price = p[markets == m]
        
        # J_t x J_t block matrix of 1's indicating goods belonging to same firm
        #  in that market/year
        # Also known as the ownership matrix
        same_firm = np.equal(firm_yr, np.transpose(firm_yr))
        
        # obtain matrix of probabilities for all simulations for goods in that 
        #  market/year
        yr = q[markets == m,:]
        
        # obtain number of goods in that market
        nobs = np.size(yr, 0)
        
        # this is the omega matrix initializing        
        grad = np.zeros((nobs, nobs))
        
        # calculate the omega matix by iterating through all individuals
        #  Omega matrix is cross-price deriv element-wise product with
        #  ownership matrix
        for i in range(N):
            yr_i = yr[:, i].reshape((-1, 1))
            grad += alpha / y[i, m - 1] * same_firm * 1/N * \
            (yr_i @ yr_i.T - np.diag(yr[:,i]))
        
        # Omega matrix actually requires negative cross price derivs
        subMatrix = -grad
        
        # now obtain the marginal costs
        b = np.linalg.inv(subMatrix) @ s[markets == m]
        mc = price - b
        mc[mc < 0] = .001
        
        # update entries in the output vector
        out[markets == m] = mc.flatten()
        
    return out

## Minimizing the GMM Objective Function

In [30]:
# If you're looking at the four steps Aviv lists in his appendix, start here
# This is the objective function that we optimize the non-linear parameters over
def objective(theta_2, s, X, V, p, y, J, T, N, marks, markets, tol, 
              Z, Z_s, W, weigh, firms):
    
    obs = np.sum(J)
    
    # Aviv's step 1 & 2:
    d.delta = solve_delta(s, X, V, p, y, d.delta, theta_2, J, T, N, tol)
    
    # obtain the actual implied quantities and shares from converged delta
    q_s = compute_share(X, V, p, y, d.delta, theta_2, J, T, N)
    
    # calculate marginal costs
    mc = calc_mc(q_s, firms, p, y, theta_2[5], J, T, N, marks, markets).reshape((-1,1))
    
    # since we are using both demand and supply side variables,
    #  we want to stack the estimated delta and estimated mc
    y2 = np.vstack((d.delta.reshape((-1,1)), np.log(mc)))
    
    # create characteristics matrix that includes both supply and demand side
    #  with demand characteristics on the bottom left and supply on the top right
    x = scipy.linalg.block_diag(X,W)
    
    # create matrix of supply and demand instruments, again with
    #  demand instruments on the right and supply on the left (top/down changed)    
    z = scipy.linalg.block_diag(Z,Z_s)
    
    # get linear parameters (this FOC is from Aviv's appendix)
    b = np.linalg.inv(x.T @ z @ weigh @ z.T @ x) @ (x.T @ z @ weigh @ z.T @ y2)
    
    # Step 3: get the error term xi (also called omega)
    xi_w = y2 - x @ b
    
    # computeo g_bar in GMM methods
    g = z.T @ xi_w / obs
    
    obj = float(obs**2 * g.T @ weigh @ g)
   
    print([theta_2, obj])
    
    return obj

In [ ]:
Z_s = np.hstack([W, generate_instrument(df, ["cons", "ln_hpwt", "air", "ln_mpg", "ln_space", "trend"], 'firm', 'market', 'firmid').values, \
                generate_instrument(df, ["cons", "ln_hpwt", "air", "ln_mpg", "ln_space", "trend"], 'market', 'market').values, \
                df.mpd.values.reshape((-1,1))])

# obtain block-diag matrix of supply and demand instruments
z = scipy.linalg.block_diag(Z,Z_s)

# Recommended initial weighting matrix from Aviv's appendix
w1 = np.linalg.inv(z.T @ z)

# Step 4: search over parameters that minimize the objective function
t1 = time.time()

# set bounds for optimization (not entirely needed but oh well)
bnds = ((0,np.inf), (0,np.inf), (0,np.inf), 
        (0,np.inf), (0,np.inf), (5,np.inf))

res_init_wt = minimize(objective,
                      theta_2, 
                      args = (df.share.values, X, V, Price, y_it, 
                              J, T, N, unique_markets, markets, 1e-4, 
                              Z, Z_s, W, w1, firms), 
                      bounds = bnds,
                      method = "L-BFGS-B",
                      options = {'maxiter': 1000, 'maxfun': 1000, 'eps': 1e-3},
                      tol = 1e-4)
    

    
time.time() - t1

[array([ 3.612,  4.628,  1.818,  1.05 ,  2.056, 43.501]), 826.0179263484974]
[array([ 3.613,  4.628,  1.818,  1.05 ,  2.056, 43.501]), 826.0745169037684]
[array([ 3.612,  4.629,  1.818,  1.05 ,  2.056, 43.501]), 826.0891615121512]
[array([ 3.612,  4.628,  1.819,  1.05 ,  2.056, 43.501]), 826.1907288321702]
[array([ 3.612,  4.628,  1.818,  1.051,  2.056, 43.501]), 826.7695885807939]
[array([ 3.612,  4.628,  1.818,  1.05 ,  2.057, 43.501]), 826.1363506639243]
[array([ 3.612,  4.628,  1.818,  1.05 ,  2.056, 43.502]), 826.134415378207]
[array([ 3.51951909,  4.50950564,  1.7714523 ,  1.02311602,  2.0033586 ,
       42.51522832]), 790.796542988201]
[array([ 3.52051909,  4.50950564,  1.7714523 ,  1.02311602,  2.0033586 ,
       42.51522832]), 790.7809428497961]
[array([ 3.51951909,  4.51050564,  1.7714523 ,  1.02311602,  2.0033586 ,
       42.51522832]), 790.7437017769572]
[array([ 3.51951909,  4.50950564,  1.7724523 ,  1.02311602,  2.0033586 ,
       42.51522832]), 790.794294168473]
[array([

In [ ]:
# save the output
outfile = open("res_init_wt_bfgs.pickle", "wb")
pickle.dump(res_init_wt, outfile)
outfile.close()

# remember to write code that reads in the above output later and comment out the save


In [ ]:
obs = np.sum(J)

# approximate optimal weighting matrix
params_2 = res_init_wt.x

# calculate mean utility given the optimal parameters (with id weighting matrix)
d.delta = solve_delta(df.share.values, X, V, p, y_it, 
                        d.delta, params_2, J, T, N, 1e-5)

# calculate probabilities and shares given the optimal params (w/ id weight matrix)
q_s = compute_share(X, V, p, y_it, d.delta, params_2, J, T, N)

# calculate marginal costs
mc = calc_mc(q_s, firms, p, y_it, params_2[5], J, T, N, marks, markets).reshape((-1,1))

y2 = np.vstack((d.delta.reshape((-1,1)), np.log(mc)))
x = scipy.linalg.block_diag(X,W)
z = scipy.linalg.block_diag(Z,Z_s)

# this is the first order condition that solves for the linear parameters
b = np.linalg.inv(x.T @ z @ w1 @ z.T @ x) @ (x.T @ z @ w1 @ z.T @ y2)

# obtain the error
xi_w = y2 - x @ b


# update weighting matrix
g_ind = z * xi_w
vg = g_ind.T @ g_ind / obs

# obtain optimal weighting matrix
weight = scipy.linalg.inv(vg)

In [ ]:
# now search for optimal parameters with the optimal weighting matrix
t1 = time.time()

res = minimize(objective,
              theta_2, 
              args = (df.share.values, X, V, p, y_it, 
                      J, T, N, marks, markets, 1e-4, 
                      Z, Z_s, W, weight, firms), 
              bounds = bnds,
              method = "L-BFGS-B",
              options = {'maxiter': 1000, 'maxfun': 1000, 'eps': 1e-3},
              tol = 1e-4)

time.time() - t1

In [ ]:
outfile = open("res_bfgs.pickle", "wb")
pickle.dump(res, outfile)
outfile.close()

In [ ]:
# obtain the linear parameters
params_3 = res.x

d.delta = solve_delta(df.share.values, X, V, p, y_it, 
                        d.delta, params_3, J, T, N, 1e-4)
q_s = compute_share(X, V, p, y_it, d.delta, params_3, J, T, N)
    
mc = calc_mc(q_s, firms, p, y_it, params_3[5], J, T, N, marks, markets).reshape((-1,1))
    
y2 = np.vstack((d.delta.reshape((-1,1)), np.log(mc)))

In [ ]:
# Table iv of BLP 1995
# first 5 are the demand side means
# last 6 are the cost side params
b = np.linalg.inv(x.T @ z @ weight @ z.T @ x) @ (x.T @ z @ weight @ z.T @ y2)
b